In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/ITESO/Eight Semester-Primavera 2024/Proyecto de ciencia de datos/Chromatic-charm/notebooks/')
from utils import make_dataloader, MainModel, train_model

import mlflow
import dagshub
import torch
from torchvision.models import inception_v3
from torcheval.metrics import FrechetInceptionDistance as FID

global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Load dataset - Places365

In [3]:
path = '/content/drive/MyDrive/ITESO/Eight Semester-Primavera 2024/Proyecto de ciencia de datos/Chromatic-charm/data/archive/val_256'

train_loader = make_dataloader(dir_path=path, split='train_samples')
print(len(train_loader))

685


In [4]:
test_loader = make_dataloader(dir_path=path, split='test_samples', batch_size=1)
print(len(test_loader))

7302


In [5]:
dagshub.init('Chromatic-charm', 'RafaJBZ', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/RafaJBZ/Chromatic-charm.mlflow')

Initialized MLflow to track repo "RafaJBZ/Chromatic-charm"

Repository RafaJBZ/Chromatic-charm initialized!

In [7]:
name_model = "GAN"
epochs = 5

parameters = {
    "lr_G" : 2e-4,
    "lr_D" : 2e-4,
    "beta1" : 0.5,
    "beta2" : 0.999,
    "lambda_L1" : 100.
}

with mlflow.start_run(run_name=name_model):

    mlflow.log_param("model", name_model)
    mlflow.log_param("epochs", epochs)
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)

    # Entrena un clasificador AdaBoost
    model = MainModel(**parameters)
    train_model(model, train_loader, epochs=epochs, checkpoints_dir='')

    # FID metric
    metric = FID()

    for data in test_loader:
        pred = model.predict(data)

        image_tensor = data['color']

        pred_normalized = torch.clamp(pred / 255.0, 0.0, 1.0)
        image_tensor_normalized = torch.clamp(image_tensor / 255.0, 0.0, 1.0)

        metric.update(image_tensor_normalized, is_real=True)
        metric.update(pred_normalized, is_real=False)

    fid_score = metric.compute()

    # Registrar métricas
    mlflow.log_metric("FID", fid_score)

    # Registrar el modelo
    mlflow.pytorch.log_model(model, name_model)


/usr/local/lib/python3.10/dist-packages/torcheval/metrics/image/fid.py:172: RuntimeWarning: Computing FID requires at least 1 real image and 1 fake image,but currently running with 0 real images and 0 fake images.Returning 0.0
  warnings.warn(
2024/05/04 00:53:56 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.2.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/05/04 00:54:03 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.4.26+cuda12.cudnn89) contains a local version label (+cuda12.cudnn89). MLflow logged a pip requirement for this package as 'jaxlib==0.4.26' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `p

In [9]:
 model.save_weights('/content/drive/MyDrive/ITESO/Eight Semester-Primavera 2024/Proyecto de ciencia de datos/Chromatic-charm/models/weight.pth')

In [10]:
name_model = "GANv2"
epochs = 5

parameters = {
    "lr_G" : 1e-4,     # Lower learning rate for generator
    "lr_D" : 3e-4,     # Higher learning rate for discriminator
    "beta1" : 0.6,     # Slightly more momentum
    "beta2" : 0.999,   # Keeping it the same
    "lambda_L1" : 50.  # Less regularization
}

with mlflow.start_run(run_name=name_model):

    mlflow.log_param("model", name_model)
    mlflow.log_param("epochs", epochs)
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)

    # Entrena un clasificador AdaBoost
    model = MainModel(**parameters)
    train_model(model, train_loader, epochs=epochs, checkpoints_dir='')

    # FID metric
    metric = FID()

    for data in test_loader:
        pred = model.predict(data)

        image_tensor = data['color']

        pred_normalized = torch.clamp(pred / 255.0, 0.0, 1.0)
        image_tensor_normalized = torch.clamp(image_tensor / 255.0, 0.0, 1.0)

        metric.update(image_tensor_normalized, is_real=True)
        metric.update(pred_normalized, is_real=False)

    fid_score = metric.compute()

    # Registrar métricas
    mlflow.log_metric("FID", fid_score)

    # Registrar el modelo
    mlflow.pytorch.log_model(model, name_model)


Output hidden; open in https://colab.research.google.com to view.

In [11]:
name_model = "GANv3"
epochs = 5

parameters = {
    "lr_G" : 3e-4,     # Higher learning rate for generator
    "lr_D" : 1e-4,     # Lower learning rate for discriminator
    "beta1" : 0.4,     # Less momentum
    "beta2" : 0.995,   # Slightly less conservative on the updates
    "lambda_L1" : 150. # More regularization
}
with mlflow.start_run(run_name=name_model):

    mlflow.log_param("model", name_model)
    mlflow.log_param("epochs", epochs)
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)

    # Entrena un clasificador AdaBoost
    model = MainModel(**parameters)
    train_model(model, train_loader, epochs=epochs, checkpoints_dir='')

    # FID metric
    inception_model = inception_v3(pretrained=True)
    inception_model.fc = torch.nn.Identity()
    inception_model.eval()

    # FID metric
    metric = FID()

    for data in test_loader:
        pred = model.predict(data)

        image_tensor = data['color']

        pred_normalized = torch.clamp(pred / 255.0, 0.0, 1.0)
        image_tensor_normalized = torch.clamp(image_tensor / 255.0, 0.0, 1.0)

        metric.update(image_tensor_normalized, is_real=True)
        metric.update(pred_normalized, is_real=False)

    fid_score = metric.compute()

    # Registrar métricas
    mlflow.log_metric("FID", fid_score)

    # Registrar el modelo
    mlflow.pytorch.log_model(model, name_model)


Output hidden; open in https://colab.research.google.com to view.